<a href="https://colab.research.google.com/github/nurcann23/AygazCompVisionBootcamp/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
#Güncelleme denemesi
import kagglehub
rrebirrth_animals_with_attributes_2_path = kagglehub.dataset_download('rrebirrth/animals-with-attributes-2')

print('Data source import complete.')

Resuming download from 969932800 bytes (12956809619 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/rrebirrth/animals-with-attributes-2?dataset_version_number=1 (969932800/13926742419) bytes left.


100%|██████████| 13.0G/13.0G [01:26<00:00, 149MB/s]

Extracting files...


Data source import complete.


In [3]:
import os

# Initialize the dictionary to hold paths for each animal
image_paths = {}

# Base directory for your dataset
base_path = "/root/.cache/kagglehub/datasets/rrebirrth/animals-with-attributes-2/versions/1/Animals_with_Attributes2/JPEGImages/"

# List of animals to search for
animals = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]

# Traverse the directory structure
for dirname, _, filenames in os.walk(base_path):
    for animal in animals:
        # Check if the current directory contains the animal's name
        if animal in dirname:
            # Initialize the list if the animal is encountered for the first time
            if animal not in image_paths:
                image_paths[animal] = []
            # Add all image paths for the current animal
            for filename in filenames:
                image_paths[animal].append(os.path.join(dirname, filename))

# Print the paths for verification
for animal, paths in image_paths.items():
    print(f"{animal}: {len(paths)} images")



giant+panda: 874 images
polar+bear: 868 images
rabbit: 1088 images
elephant: 1038 images
collie: 1028 images
squirrel: 1200 images
sheep: 1420 images
fox: 664 images
moose: 704 images
dolphin: 946 images


In [5]:
!pip install opencv-contrib-python


In [7]:
import tensorflow as tf

image_size = (128, 128)  # İstenen boyut
normalized_data = []

with tf.device('/GPU:0'):
    for cls, paths in image_paths.items():
        for img_path in paths:
            img_raw = tf.io.read_file(img_path)  # Resmi yükle
            img_tensor = tf.image.decode_jpeg(img_raw, channels=3)  # Resmi çöz
            img_resized = tf.image.resize(img_tensor, image_size)  # Yeniden boyutlandır
            img_normalized = img_resized / 255.0  # Normalize et
            normalized_data.append((img_normalized, cls))

print("Tüm resimler başarıyla normalize edildi.")


Tüm resimler başarıyla normalize edildi.


In [10]:
import tensorflow as tf
print("GPU kullanılabilir mi:", tf.config.list_physical_devices('GPU'))


GPU kullanılabilir mi: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [24]:
# İlk 5 normalleştirilmiş resmi ve etiketlerini göster
for i, (img, label) in enumerate(normalized_data[:5]):
    print(f"Resim {i + 1} - Etiket: {label}, Boyut: {img.shape}")


Resim 1 - Etiket: giant+panda, Boyut: (128, 128, 3)
Resim 2 - Etiket: giant+panda, Boyut: (128, 128, 3)
Resim 3 - Etiket: giant+panda, Boyut: (128, 128, 3)
Resim 4 - Etiket: giant+panda, Boyut: (128, 128, 3)
Resim 5 - Etiket: giant+panda, Boyut: (128, 128, 3)


In [25]:
from sklearn.model_selection import train_test_split
# Resimleri ve etiketleri ayır
X = [x[0] for x in normalized_data]
y = [x[1] for x in normalized_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("Eğitim ve test setleri başarıyla oluşturuldu.")
print(f"Eğitim setindeki resim sayısı: {len(X_train)}")
print(f"Test setindeki resim sayısı: {len(X_test)}")
print(f"Eğitim setindeki etiket sayısı: {len(y_train)}")
print(f"Test setindeki etiket sayısı: {len(y_test)}")

Eğitim ve test setleri başarıyla oluşturuldu.
Eğitim setindeki resim sayısı: 6881
Test setindeki resim sayısı: 2949
Eğitim setindeki etiket sayısı: 6881
Test setindeki etiket sayısı: 2949


In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Modeli oluştur
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),  # Daha küçük filtre
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),  # Daha küçük filtre
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),  # Daha az nöron
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Modeli derle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 126, 126, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 63, 63, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 61, 61, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 256)                 │      29,491,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,569,674 (112.80 MB)

 Trainable params: 29,569,674 (112.80 MB)

 Non-trainable params: 0 (0.00 B)

None


In [29]:
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

# Etiketleri one-hot encode
# Assuming 'animals' is your list of all classes
class_mapping = {cls: i for i, cls in enumerate(animals)}
y_train_enc = to_categorical([class_mapping[label] for label in y_train], num_classes=10)
y_test_enc = to_categorical([class_mapping[label] for label in y_test], num_classes=10)
# Modeli eğit

#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(np.array(X_train), np.array(y_train_enc), epochs=10, validation_data=(np.array(X_test), np.array(y_test_enc)))


Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - accuracy: 0.2619 - loss: 2.3643 - val_accuracy: 0.4836 - val_loss: 1.6197
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.5076 - loss: 1.4799 - val_accuracy: 0.5310 - val_loss: 1.3938
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.5839 - loss: 1.2175 - val_accuracy: 0.5585 - val_loss: 1.3079
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.6804 - loss: 0.9401 - val_accuracy: 0.5704 - val_loss: 1.2883
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.7492 - loss: 0.7338 - val_accuracy: 0.5534 - val_loss: 1.3906
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.8167 - loss: 0.5574 - val_accuracy: 0.5571 - val_loss: 1.4892
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.8648 - loss: 0.4097 - val_accuracy: 0.5310 - val_loss: 1.8564
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 8s 38ms/step - accuracy: 0.8527 - loss: 0.4894 - val_